In [ ]:
import sys
sys.path.append('..')

from .tsne import visualize_tsne
import torch.nn as nn
import torch.nn.functional as F
import torch
import argparse
from torch.utils import data
import numpy as np
import random
from tqdm import tqdm
from config import get_config

In [ ]:
def load_embedding(path):
    emb = torch.load(path)['embedding']
    emb = list(map(lambda x: x.cpu()[0], emb))
    return emb  

In [ ]:
def get_embedding(src_embeddings):
    src_embeddings = torch.stack(src_embeddings).to('cpu')
    img_emb_src = src_embeddings.mean(dim=0, keepdim=True)
    img_emb_src /= img_emb_src.norm(dim=-1, keepdim=True)
    img_emb_src = img_emb_src.repeat(1,1).type(torch.float32)  # (B,1024)
    print(img_emb_src.shape)
    return img_emb_src.cpu()

In [ ]:
def norm_embeddings(src_embeddings):
    src_embeddings = src_embeddings.clone()
    for i in range(len(src_embeddings)):
        src_embeddings[i] /= src_embeddings[i].norm(dim=-1, keepdim=True)
    return torch.stack(src_embeddings)

In [ ]:
def normalize(embeddings): #list of list of embds
    results = []
    for lst in embeddings:
        res = []
        for emb in lst:
            emb = emb.clone()
            emb /= emb.norm(dim=-1, keepdim=True)
            res.append(emb)
        results.append(res)
    return results

In [ ]:
device = torch.device('cuda:2')
CORRUPTIONS = ["gaussian_noise", "shot_noise", "impulse_noise", "defocus_blur", "glass_blur", "motion_blur", "zoom_blur", "snow", "frost", "fog", "brightness", "contrast", "elastic_transform", "pixelate", "jpeg_compression"]

In [ ]:
src_embeddings = load_embedding('../data/embeddings/embedding_cifar10_wideresnet28_640avgn.pth')
target_embeddings = load_embedding('../data/embeddings/embedding_cifar10c_wideresnet28.pth')

print(f'{len(src_embeddings)=}, {len(target_embeddings)=}')
print(f'{src_embeddings[0].shape=}, {target_embeddings[0].shape=}')